<a href="https://colab.research.google.com/github/L20660046/Actividad-1/blob/main/Actividad3_GUI_Salarios_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 3 — GUI Modelo de Predicción de Salarios




In [ ]:
# @title Instalar dependencias
!pip -q install gradio pandas numpy tensorflow


In [1]:
# @title Backend: modelo, utilidades y 'base de datos' CSV
from __future__ import annotations
import os, math, uuid, datetime as _dt
import numpy as np
import pandas as pd
import tensorflow as tf
import gradio as gr

MODEL_PATH = "/content/model_7.keras"   # cambia a tu ruta si usas Drive
DB_PATH    = "/content/salary_records.csv"  # 'Base de datos' local (CSV)

QUALS = ["BSc", "MSc", "PhD"]
ROLES = ["Junior", "Mid", "Senior"]
UNIS  = ["Tier1", "Tier2", "Tier3"]

def _exists(p):
    try: return os.path.exists(p)
    except Exception: return False

def _ensure_db():
    if not _exists(DB_PATH):
        cols = [
            'id','timestamp','name','phone','dob','experience','qualification','role',
            'cert','university','predicted_annual','predicted_monthly','model'
        ]
        pd.DataFrame(columns=cols).to_csv(DB_PATH, index=False)

def load_db():
    _ensure_db()
    try: return pd.read_csv(DB_PATH)
    except Exception:
        return pd.DataFrame()

def save_db(df: pd.DataFrame):
    df.to_csv(DB_PATH, index=False)

def next_id(df):
    if df.empty: return 1
    try:
        return int(df['id'].max()) + 1
    except Exception:
        return 1

def build_dummy_model(save_path: str):
    # Recibe vector de 12 features (3 num + 9 one-hot)
    rng = np.random.default_rng(42)
    N = 4000
    exp = rng.integers(0, 21, size=N).astype(np.float32)
    age = rng.integers(20, 61, size=N).astype(np.float32)
    cert = rng.integers(0, 2, size=N).astype(np.float32)
    qi = rng.integers(0, 3, size=N)
    ui = rng.integers(0, 3, size=N)
    ri = rng.integers(0, 3, size=N)
    def oh(idx):
        m = np.zeros((N,3), dtype=np.float32)
        m[np.arange(N), idx] = 1.0
        return m
    X = np.column_stack([exp, age, cert, oh(qi), oh(ui), oh(ri)])
    base = 40000
    role_bonus = np.array([0, 15000, 30000], dtype=np.float32)[ri]
    qual_bonus = np.array([0, 10000, 20000], dtype=np.float32)[qi]
    uni_bonus  = np.array([8000, 4000, 0], dtype=np.float32)[ui]
    y = base + 3000*exp + role_bonus + qual_bonus + uni_bonus + 3000*cert
    y += rng.normal(0, 2000, size=N).astype(np.float32)
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(12,)),
        tf.keras.layers.Dense(48, activation='relu'),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=10, batch_size=64, verbose=0)
    model.save(save_path)
    return model

def one_hot(value: str, vocab: list[str]) -> list[float]:
    vec = [0.0]*len(vocab)
    if value in vocab:
        vec[vocab.index(value)] = 1.0
    return vec

def parse_date_any(text: str) -> str:
    """Devuelve 'YYYY-MM-DD' desde 'DD/MM/AAAA' o ISO."""
    if not text: return ''
    try:
        # Primero intentamos DD/MM/AAAA
        d = pd.to_datetime(text, dayfirst=True, errors='coerce')
        if pd.isna(d):
            d = pd.to_datetime(text, errors='coerce')
        return str(d.date())
    except Exception:
        return ''

def age_from_iso(date_iso: str) -> float:
    if not date_iso: return 0.0
    try:
        y, m, d = map(int, date_iso.split('-'))
        dob = _dt.date(y, m, d)
        today = _dt.date.today()
        return float(today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day)))
    except Exception:
        return 0.0

def format_phone(number: str) -> str:
    digits = ''.join(ch for ch in str(number) if ch.isdigit())
    return digits

# Cargar o crear modelo
if _exists(MODEL_PATH):
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print("✅ Modelo cargado:", MODEL_PATH)
    except Exception:
        print("❌ No se pudo cargar. Se creará un modelo base…")
        model = build_dummy_model(MODEL_PATH)
else:
    print("ℹ️ No hay modelo. Creando modelo base…")
    model = build_dummy_model(MODEL_PATH)
    print("✅ Modelo base guardado en:", MODEL_PATH)

MODEL_TAG = "Base del Modelo 7" if "model_7" in os.path.basename(MODEL_PATH) else f"Modelo: {os.path.basename(MODEL_PATH)}"

def predict_raw(experience: float, age: float, qual: str, uni: str, role: str, cert: str) -> float:
    cert_bin = 1.0 if str(cert).strip().lower() == 'yes' else 0.0
    x = [float(experience), float(age), float(cert_bin)]
    x += one_hot(qual, QUALS)
    x += one_hot(uni, UNIS)
    x += one_hot(role, ROLES)
    x = np.array(x, dtype='float32').reshape(1, -1)
    pred = model.predict(x, verbose=0)
    return float(np.ravel(pred)[0])

def predict_and_save(name, phone, dob_text, exp, qual, role, cert, uni, period):
    dob_iso = parse_date_any(dob_text)
    age = age_from_iso(dob_iso)
    annual = predict_raw(exp, age, qual, uni, role, cert)
    monthly = annual / 12.0
    show = monthly if period == 'Mensual' else annual
    # Guardar en CSV
    df = load_db()
    rid = next_id(df)
    new_row = {
        'id': rid,
        'timestamp': _dt.datetime.now().isoformat(timespec='seconds'),
        'name': name,
        'phone': format_phone(phone),
        'dob': dob_iso,
        'experience': exp,
        'qualification': qual,
        'role': role,
        'cert': cert,
        'university': uni,
        'predicted_annual': round(annual, 2),
        'predicted_monthly': round(monthly, 2),
        'model': MODEL_TAG,
    }
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    save_db(df)
    return (
        f"${show:,.2f}",
        f"{MODEL_TAG}",
        f"Registro guardado con ID #{rid}",
        f"Consejo: revisa teléfono (10 dígitos) y fecha (DD/MM/AAAA) para trazabilidad."
    )

def clear_form():
    return ("Austin Powers", "9031211691", "13/03/1992", 2, "MSc", "Mid", "Yes", "Tier1", "Anual")

def search_records(query: str):
    df = load_db()
    if not query:
        return df, sorted(df['id'].astype(int).tolist()) if not df.empty else []
    q = str(query).lower()
    mask = df.apply(lambda r: q in str(r['name']).lower() or q in str(r['phone']), axis=1)
    fdf = df[mask]
    return fdf, sorted(fdf['id'].astype(int).tolist()) if not fdf.empty else []

def delete_record(rec_id):
    if rec_id is None: return load_db(), "Selecciona un ID"
    df = load_db()
    before = len(df)
    df = df[df['id'] != int(rec_id)]
    save_db(df)
    after = len(df)
    msg = f"Eliminado ID #{rec_id}" if after < before else f"ID #{rec_id} no encontrado"
    return df, msg

def predict_for_csv(file_obj):
    if file_obj is None: return pd.DataFrame()
    df = pd.read_csv(file_obj.name)
    out = []
    for _, r in df.iterrows():
        dob_iso = parse_date_any(str(r.get('Date_Of_Birth','')))
        age = age_from_iso(dob_iso)
        val = predict_raw(
            float(r.get('Experience', 0)),
            float(age),
            str(r.get('Qualification','BSc')),
            str(r.get('University','Tier1')),
            str(r.get('Role','Junior')),
            str(r.get('Cert','No')),
        )
        out.append(val)
    df['Predicted_Salary'] = out
    return df


ℹ️ No hay modelo. Creando modelo base…
✅ Modelo base guardado en: /content/model_7.keras


In [3]:
# @title UI Gradio — estilo oscuro + pestañas (FIX eliminar/buscar)
DARK_CSS = """
.gradio-container {background: #0b1220; color: #e8eefc}
h1, h2, h3, h4, h5 {color:#e8eefc}
.card {background:#131c2b; border:1px solid #1f2a44; border-radius:16px; padding:16px}
.muted {color:#a0aac8}
.badge {display:inline-block; padding:6px 10px; border-radius:999px; background:rgba(167,139,250,.12); color:#c4b5fd; border:1px solid rgba(167,139,250,.35)}
.sep {height:2px; background:#1f2a44; border-radius:2px; margin:8px 0 16px}
.result-number {font-size:28px; font-weight:700}
.gr-button-primary {border-radius:12px}
"""

# ----- Wrappers para actualizar correctamente el Dropdown de IDs -----
def _ids_update_from_df(df):
    """Devuelve gr.update(choices=[...], value=None) según los IDs del DF."""
    if df is None or df.empty:
        return gr.update(choices=[], value=None)
    ids = pd.to_numeric(df.get('id'), errors='coerce').dropna().astype(int).tolist()
    ids = sorted(set(ids))
    return gr.update(choices=[str(i) for i in ids], value=None)

def _wrap_search(q: str):
    # search_records puede devolver (df, lista_ids), pero recomputamos por seguridad.
    df, _ = search_records(q)
    return df, _ids_update_from_df(df)

def _wrap_init():
    df, _ = search_records("")
    return df, _ids_update_from_df(df)

def _wrap_delete(rid):
    # delete_record devuelve (df, msg). Luego refrescamos las choices del Dropdown.
    df, msg = delete_record(rid)
    return df, _ids_update_from_df(df), msg
# --------------------------------------------------------------------

def build_ui():
    with gr.Blocks(title="Predictor de Salario", theme=gr.themes.Soft(), css=DARK_CSS) as demo:
        gr.Markdown("""
        # **Predictor de Salario**
        <span class='muted'>Mercantil de la Rosa SA de CV · Modelo DNN (UI Demo) · Integrantes: Jesus Abdiel Chapa Cruz, Juan Abisai Landeros Rodriguez</span>
        """)

        with gr.Tabs():
            with gr.Tab("🟩 Predecir & Guardar"):
                with gr.Row():
                    with gr.Column(scale=6):
                        gr.Markdown("<div class='card'><h3>Datos del candidato</h3>")
                        name = gr.Textbox(label="Nombre del Empleado", value="Austin Powers", placeholder="Ej. Austin Powers")
                        phone = gr.Textbox(label="Número Telefónico (10 dígitos)", value="9031211691")
                        dob = gr.Textbox(label="Fecha de Nacimiento (DD/MM/AAAA)", value="13/03/1992")
                        exp = gr.Slider(label="Años de experiencia", minimum=0, maximum=40, step=1, value=2)
                        qual = gr.Dropdown(QUALS, value="MSc", label="Qualification Level (*)")
                        role = gr.Dropdown(ROLES, value="Mid", label="Rol Solicitado (*)")
                        cert = gr.Dropdown(["Yes","No"], value="Yes", label="Certificado (*)")
                        uni  = gr.Dropdown(UNIS, value="Tier1", label="University Tier (*)")
                        period = gr.Radio(["Anual","Mensual"], value="Anual", label="Periodo:")
                        with gr.Row():
                            btn_calc = gr.Button("Calcular Salario Óptimo", variant="primary")
                            btn_clear = gr.Button("Limpiar formulario")
                        gr.Markdown("</div>")

                    with gr.Column(scale=5):
                        gr.Markdown("<div class='card'><h3>Resultado</h3>")
                        salary_md = gr.Markdown("<div class='result-number'>—</div>")
                        model_badge = gr.Markdown("<span class='badge'>Base del Modelo 7</span>")
                        status_md = gr.Markdown(visible=True)
                        tip_md = gr.Markdown("<span class='muted'>Consejo: revisa teléfono (10 dígitos) y fecha (DD/MM/AAAA) para trazabilidad.</span>")
                        gr.Markdown("</div>")

                btn_calc.click(
                    predict_and_save,
                    inputs=[name, phone, dob, exp, qual, role, cert, uni, period],
                    outputs=[salary_md, model_badge, status_md, tip_md]
                )
                btn_clear.click(
                    clear_form,
                    outputs=[name, phone, dob, exp, qual, role, cert, uni, period]
                )

            with gr.Tab("🔎 Buscar / 🗑 Eliminar"):
                with gr.Row():
                    query = gr.Textbox(label="Buscar por nombre o teléfono")
                with gr.Row():
                    df_view = gr.Dataframe(label="Resultados", interactive=False)
                with gr.Row():
                    id_drop = gr.Dropdown(choices=[], label="ID a eliminar")
                    btn_del = gr.Button("Eliminar seleccionado")
                    del_status = gr.Markdown()

                # FIX: usar wrappers que devuelven gr.update(...) para el Dropdown
                query.change(_wrap_search, inputs=query, outputs=[df_view, id_drop])
                btn_del.click(_wrap_delete, inputs=id_drop, outputs=[df_view, id_drop, del_status])

                # Carga inicial correcta (sin asignar .value/.choices manualmente)
                demo.load(_wrap_init, outputs=[df_view, id_drop])

            with gr.Tab("✅ Validar contra CSV"):
                gr.Markdown("Sube `new_hires.csv` (Name, Phone_Number, Experience, Qualification, University, Role, Cert, Date_Of_Birth)")
                csv_in = gr.File(label="Subir CSV", file_types=[".csv"], interactive=True)
                df_out = gr.Dataframe(label="Predicciones", interactive=False)
                csv_in.change(predict_for_csv, inputs=csv_in, outputs=df_out)

    return demo

app = build_ui()
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://197bc987da2092dd79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
